In [129]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import numpy as np
import collections as col
import scipy.sparse as sp
from scipy.sparse import lil_matrix
import scipy.sparse.linalg as spl

[nltk_data] Downloading package stopwords to C:\Users\Zineb
[nltk_data]     BENISS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [228]:
nb_doc= 20
data_test = pd.read_csv("./arxiv_articles.csv", sep="|",usecols=[1,3,4],nrows = nb_doc)
#nrows permet de selectionner un certain nb de docs 

In [229]:
def f(arg): pass
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
#fonctions appliquées sur chaque document pour nettoyage
#b = lambda new : " ".join([ps.stem(i) for i in re.sub("[^a-zA-Z]", " " ,new.lower()).split() if i not in stop_words]).split()
#a = lambda x : col.Counter(re.findall('\w+',x))
#j'ai enlevé le stemmer et ya encore qques lettres qui trainent
b = lambda new : " ".join([i for i in re.sub("[^a-zA-Z]", " " ,new.lower()).split() if i not in stop_words]).split()

In [230]:
data_test['cleaned']=data_test['summary'].apply(b)
#data_test['dict']=data_test['cleaned'].apply(a)

In [231]:
def calcul_tf(d):
    term_counts = pd.Series(d).value_counts()
    max_mot = np.max(term_counts)
    return 0.5 + 0.5 * (term_counts / max_mot)
def compute_idf(D):
    N = len(D)
    all_terms = pd.concat(D)
    nt = all_terms.index.value_counts() # The number of documents containing the term "t"
    return np.log(N / nt)
def compute_tf_idf_document(tf_document, idf):
    return tf_document * np.array([idf[i] for i in tf_document.index])
#calcul le tf idf pour 1 document
def compute_tf_idf_corpus(D):
    term_freq = [calcul_tf(d) for d in D]
    idf = compute_idf(term_freq)
    return [compute_tf_idf_document(d, idf) for d in term_freq]
#prend en entree une colonne panda series et return un 

In [232]:
tf_idf = compute_tf_idf_corpus(data_test.loc[:10000,"cleaned"])
#liste avec comme indice le numéro du document et pour chaque document un panda series avec mot comme index et tf_idf pour
#le mot comme values array de nos mots mais ne les mets pas en ordre (a supprimer)
all_mot = pd.concat(tf_idf).index.drop_duplicates() 
#print(tf_idf)

In [233]:
data=pd.DataFrame(tf_idf).fillna(0)
dense_matrix = np.array(data.as_matrix())
mots=list(data)
matrix = sp.csr_matrix(dense_matrix)

In [234]:
#distance entre 2 docs
print(spl.norm(matrix[0]-matrix[1]))

15.02517767982351


In [275]:
class K_Means:

    def __init__(self, k =5, tolerance = 0.001, max_iterations = 50):
        self.k = k
        self.tolerance = tolerance
        self.max_iterations = max_iterations
        
    #data sera notre matrice sparse
    def fit(self, data):

        self.centroids = {}
        
        #initialize the centroids, the first 'k' elements in the dataset will be our initial centroids (à changer?)
        for i in range(self.k):
            self.centroids[i] = data[i]

        #begin iterations
        for i in range(self.max_iterations):
            self.classes = {}
            #l'attribut self.classes contient pour chaque sujet les vecteurs qui lui sont le plus proche grâce au append
            for i in range(self.k):
                self.classes[i] = []

            #find the distance between the point and cluster; choose the nearest centroid
            for doc in data:
                distances = [spl.norm(doc - self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classes[classification].append(doc)
            
            #garde en mémoire les derniers centroids
            previous = dict(self.centroids)
            #average the cluster datapoints to re-calculate the centroids
            for classification in self.classes:
                self.centroids[classification] = sp.csr_matrix.mean(self.classes[classification][0], axis = 0)

            isOptimal = True

            for centroid in self.centroids:

                original_centroid = previous[centroid]
                curr = self.centroids[centroid]

                if sp.csr_matrix.sum((curr - original_centroid)/original_centroid * 100.0) > self.tolerance:
                    isOptimal = False

            #break out of the main loop if the results are optimal, ie. the centroids don't change their positions much(more than our tolerance)
            if isOptimal:
                break

    def pred(self, data):
        distances = [sp.linalg.norm(data - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification


In [276]:
km = K_Means()
km.fit(matrix)

   

TypeError: unsupported operand type(s) for /: 'matrix' and 'csr_matrix'

# test

In [283]:
data = matrix
K=5
iteration =1
centroids  = {}
tolerance = 0.01


for i in range(K):
    centroids[i] = data[i]
    
    
for i in range(iteration):
    classes = {}
    for j in range(K):
        classes[j] = []
        
    #for i in range(nb_doc) : 
    for i in data : 
        distances = [spl.norm(i - centroids[centroid]) for centroid in centroids]
        classification = distances.index(min(distances))
        classes[classification].append(i)

    previous = dict(centroids)
                         
    for classif in classes:
        centroids[classification] = sp.csr_matrix.mean(classes[classification][0], axis = 0)
        
    isOptimal = True
    
    for centroid in centroids:
        original_centroid = previous[centroid]
        curr = centroids[centroid]
    
        if sp.csr_matrix.sum((curr - original_centroid)/original_centroid * 100.0) > self.tolerance:
                isOptimal = False
    

AttributeError: 'matrix' object has no attribute '_swap'

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57

In [137]:
print(matrix

  (0, 0)	1.0729586082894
  (0, 2)	1.34119826036175
  (0, 3)	1.0729586082894
  (0, 4)	1.0729586082894
  (0, 9)	1.0729586082894
  (0, 10)	1.0729586082894
  (0, 13)	1.34119826036175
  (0, 21)	1.34119826036175
  (0, 22)	1.6094379124341003
  (0, 23)	1.34119826036175
  (0, 34)	0.7635756098951292
  (0, 36)	1.0729586082894
  (0, 37)	1.34119826036175
  (0, 38)	1.0729586082894
  (0, 39)	1.0729586082894
  (0, 43)	1.34119826036175
  (0, 49)	1.0729586082894
  (0, 50)	1.0729586082894
  (0, 52)	1.0729586082894
  (0, 55)	1.6094379124341003
  (0, 58)	0.6108604879161034
  (0, 62)	1.34119826036175
  (0, 63)	1.0729586082894
  (0, 67)	0.6108604879161034
  (0, 68)	1.0729586082894
  :	:
  (3, 15)	1.34119826036175
  (3, 16)	1.0729586082894
  (3, 17)	1.0729586082894
  (3, 19)	1.0729586082894
  (3, 26)	1.0729586082894
  (3, 28)	1.34119826036175
  (3, 42)	1.0729586082894
  (3, 45)	1.0729586082894
  (3, 46)	1.34119826036175
  (3, 47)	1.34119826036175
  (3, 51)	1.0729586082894
  (3, 54)	1.6094379124341003
  (3, 57